In [1]:
import pandas as pd
data = pd.read_csv('data_000637.txt')
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
def selection(N):
    if ((N < data.index.stop) & (N>10000)):
        print(data.loc[0:N])
    else:
        print("Value N is not valid")

# numero di righe = data.index.stop

In [3]:
selection(200500)

        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          124  3869200167        2374        27
2          1     0           63  3869200167        2553        28
3          1     0           64  3869200167        2558        19
4          1     0           64  3869200167        2760        25
...      ...   ...          ...         ...         ...       ...
200496     1     0           51  3869202161        2430         9
200497     1     1           19  3869202161        2425        16
200498     1     1            1  3869202161        2427         4
200499     1     0           53  3869202161        2428        10
200500     1     1           11  3869202161        2426        15

[200501 rows x 6 columns]


2. Estimate the number of BX in a ORBIT (the value 'x').

In [4]:
uniques = data.T.apply(lambda x: x.nunique(), axis=1)
max_BX = uniques[4]
print('There are', max_BX, 'BX in an ORBIT')

There are 3564 BX in an ORBIT


3. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [5]:
import datetime

nrow = 0
row_initial = data.iloc[nrow]
seconds_initial = (row_initial['TDC_MEAS']*25/30 + row_initial['BX_COUNTER']*25 + row_initial['ORBIT_CNT']*(max_BX)*25)*10**(-9)

print(row_initial)
print('\nDuration represented by row number', nrow, ':')
print(str(datetime.timedelta(seconds = seconds_initial)))

HEAD                    1
FPGA                    0
TDC_CHANNEL           123
ORBIT_CNT      3869200167
BX_COUNTER           2374
TDC_MEAS               26
Name: 0, dtype: int64

Duration represented by row number 0 :
3 days, 23:45:45.734939


4. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [6]:
data['Absolute Time (ns)'] = (data['TDC_MEAS']*25/30 + data['BX_COUNTER']*25 + data['ORBIT_CNT']*(max_BX)) * (10**(-9))
data['Absolute Time (ns)']

0          13789.829455
1          13789.829455
2          13789.829459
3          13789.829459
4          13789.829464
               ...     
1310715    13789.868633
1310716    13789.868633
1310717    13789.868633
1310718    13789.868633
1310719    13789.868633
Name: Absolute Time (ns), Length: 1310720, dtype: float64

5. Use the .groupby() method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [7]:
#since HEAD column is of all 1's
#i can use it as a counter for each TDC_CHANNEL
data.groupby(['TDC_CHANNEL']).count()['HEAD'].sort_values(ascending = False)[0:3]

TDC_CHANNEL
139    108059
64      66020
63      64642
Name: HEAD, dtype: int64

6. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [71]:
df_wNan = data[data>0]
df_wNan.isna()
df_wNan.fillna(value=0)

empty = data.loc[data['ORBIT_CNT'] == 0]

In [72]:
empty

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Absolute Time (ns)


In [62]:
# it means that there are no non-empty orbits

In [78]:
orbits_in_139 = data.loc[data['TDC_CHANNEL'] == 139]
uniques_in_139 = data.T.apply(lambda x: x.nunique(), axis=1)
uniques_in_139['ORBIT_CNT']

11001